In [2]:
import os
import PyPDF2
import requests
from pprint import pprint

In [3]:
def ask_ollama(prompt):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "deepseek-r1",
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, json=payload)
    result = response.json()
    return result['response']

print(ask_ollama("How cold is too cold to jog?"))


<think>
Okay, so I'm trying to figure out how cold it's too cold to jog. I know that exercise is good for you in general, but when the weather gets really cold, I wonder if I should avoid going out for a run or bike ride. Let me think about this.

First, I guess the main factors are comfort and health. If it's extremely cold, my body might not be able to generate enough heat through exercise, which could be bad for my muscles or heart. But how cold is too cold? I'm not sure what the exact temperatures are, but I know that below freezing can be a problem.

I've heard that in the winter, people often use jackets and layers when exercising. Maybe that's why even if it's cold outside, you still go jogging inside if it's below a certain temperature. But how does that translate to outdoor activities?

Then there are the health aspects. I remember reading somewhere that too cold can lead to cramps or even frostbite, which is really painful and can be dangerous. So maybe anything below 30°F (a

In [40]:
filenames = os.listdir('data/pdf')
papers = dict()

for filename in filenames:
    texts = []
    with open(f'data/pdf/{filename}', 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()
            texts.append(text)

    full_text = '\n'.join(texts)
    papers[filename] = full_text
print(papers.keys())


dict_keys(['building_astrobert.pdf', 'Attention.pdf', 'gravitational_duals.pdf', 'BERT.pdf'])


In [13]:
with open('prompts/abstract_to_claims_prompt.txt', 'r') as file:
    template = file.read()
prompt = template.format(text=papers['BERT.pdf'][:1306])
print(prompt)

I have a research paper (or abstract) that I would like you to analyze. Your task is to extract and clearly list the paper's original results, claims, contributions, or innovations. Please focus on key points that highlight what is novel or significant about the research, avoiding general background information or restatements of existing knowledge.

Provide the results/claims/contributions/innovations as a JSON string array, using concise and precise language. Ensure each item reflects the specific value the paper adds to its field.

Example abstract:
"""
We introduce LLaMA, a collection of foundation language models ranging from 7B to 65B parameters. We train our models on trillions of tokens, and show that it is possible to train state-of-the-art models using publicly available datasets exclusively, without resorting to proprietary and inaccessible datasets. In particular, LLaMA-13B outperforms GPT-3 (175B) on most benchmarks, and LLaMA-65B is competitive with the best models, Chinc

In [17]:
response = ask_ollama(prompt)
print(response)

<think>
Alright, I need to analyze the provided research paper abstract about BERT and extract its original results, claims, contributions, or innovations. Let's break it down step by step.

First, the abstract introduces BERT as a new language representation model. It mentions that unlike previous models like those from Peters et al., Radford et al., BERT pre-trains deep bidirectional representations using unlabeled text. This joint conditioning on both left and right context in all layers is key. So, this is a significant innovation because it's simpler but more effective.

Next, the abstract states that BERT can be fine-tuned with just an additional output layer for various tasks like question answering and NLP tasks without major changes. That's a clear contribution—it makes pre-training efficient and adaptable to different models.

Then, it highlights BERT's performance improvements on several benchmarks. It mentions pushing the GLUE score by 7.7%, improving MultiNLI accuracy by 4

In [18]:
# For Deepseek-r1, the output has a <think>...</think> section we can trim
def postprocess_deepseek(response):
    return response.split('</think>')[1].strip()

print(postprocess_deepseek(response))

```json
[
  "Introduction of BERT, a new language representation model that pre-trains deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers.",
  "BERT can be fine-tuned with just one additional output layer to create state-of-the-art models for tasks such as question answering and language understanding without substantial task-specific architecture modifications.",
  "BERT achieves significant performance improvements on several natural language processing benchmarks, including a 7.7% absolute improvement in the GLUE score, a 4.6% absolute improvement in MultiNLI accuracy, a 1.5 point absolute improvement in SQuAD v1.1 Test F1, and a 5.1 point absolute improvement in SQuAD v2.0 Test F1.",
  "BERT demonstrates that it is possible to achieve state-of-the-art performance on these tasks without relying on expensive or inaccessible datasets."
]
```


In [48]:
prompt = template.format(text=papers['Attention.pdf'][587:1737])
print(postprocess_deepseek(ask_ollama(prompt)))

The abstract introduces a new Transformer model that replaces traditional encoder-decoder architectures with solely attention mechanisms. This innovation allows for more efficient training, faster convergence, and improved performance in machine translation tasks compared to existing models like ensembles from literature. The model achieves state-of-the-art results on two translation benchmarks (WMT 2014 English-to-German and English-to-French) while requiring less training time. Additionally, it demonstrates versatility by successfully applying to English constituency parsing with both ample and limited data.

```json
[
    "Introducing the Transformer architecture, which replaces traditional encoder-decoder architectures with solely attention mechanisms.",
    "Superior performance in machine translation tasks compared to existing models like ensembles from literature.",
    "Achieves state-of-the-art single-model BLEU scores of 28.4 and 41.8 for English-to-German and English-to-Fren

In [54]:
prompt = template.format(text=papers['building_astrobert.pdf'][513:1427])
print(postprocess_deepseek(ask_ollama(prompt)))

[
  "Introduction of astroBERT, a deeply contextual language model trained on recent astronomy publications.",
  "Application of machine learning and natural language processing techniques to enhance NASA's Astrophysics Data System (ADS) for semantic search.",
  "Development of a named entity recognition tool within the context of astronomical datasets to improve discoverability.",
  "Use of astroBERT to address ambiguity in search queries, such as distinguishing between various meanings of terms like 'Planck'."
]


In [68]:
prompt = template.format(text=papers['gravitational_duals.pdf'][846:1653])
print(postprocess_deepseek(ask_ollama(prompt)))

```json
[
  "Introduction of a novel method using physics-informed neural networks to solve an inverse problem in holography",
  "Development of an algorithm that is both data-driven and informed by Einstein's equations for determining gravitational theories from a prescribed equation of state",
  "Successful application of the approach to theories involving crossovers, first-order phase transitions, and second-order phase transitions"
]
```
